## imports

In [45]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


## getting the links for each laptop

In [46]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# Set up Selenium WebDriver
driver = webdriver.Chrome()

# Function to extract product links from a page
def extract_product_links(page_url):
    driver.get(page_url)
    time.sleep(5)  # Wait for the page to load
    
    product_links = set()
    
    # Extract product links from the current page
    products = driver.find_elements(By.CSS_SELECTOR, "a.product-item-link")
    for product in products:
        link = product.get_attribute("href")
        if link:
            product_links.add(link)
    
    return product_links

# Base URL and maximum number of pages to scrape
base_url = "https://goldentech.com.sa/en/laptop.html?p="
max_page = 20  # You can modify this if you need more pages

# Set to store all unique product links
all_product_links = set()

# Loop through all the pages from 1 to max_page
for page_num in range(1, max_page + 1):
    page_url = base_url + str(page_num)
    print(f"Scraping page {page_num}: {page_url}")
    
    # Extract product links for the current page
    page_product_links = extract_product_links(page_url)
    
    # Add the extracted links to the main set
    all_product_links.update(page_product_links)

# Print the number of unique product links found
print(f"Found {len(all_product_links)} unique product links.")

# Close the driver
driver.quit()

Scraping page 1: https://goldentech.com.sa/en/laptop.html?p=1
Scraping page 2: https://goldentech.com.sa/en/laptop.html?p=2
Scraping page 3: https://goldentech.com.sa/en/laptop.html?p=3
Scraping page 4: https://goldentech.com.sa/en/laptop.html?p=4
Scraping page 5: https://goldentech.com.sa/en/laptop.html?p=5
Scraping page 6: https://goldentech.com.sa/en/laptop.html?p=6
Scraping page 7: https://goldentech.com.sa/en/laptop.html?p=7
Scraping page 8: https://goldentech.com.sa/en/laptop.html?p=8
Scraping page 9: https://goldentech.com.sa/en/laptop.html?p=9


KeyboardInterrupt: 

In [30]:
# Convert product links set to a DataFrame
product_links_df = pd.DataFrame(list(all_product_links), columns=["Product Links"])

In [24]:
# Save the DataFrame to a CSV file
product_links_df.to_csv("product_linkss.csv", index=False)

In [27]:
# Print confirmation
print(f"Saved {len(all_product_links)} product links to 'product_links.csv'")

Saved 315 product links to 'product_links.csv'


In [48]:
product_links_df = pd.read_csv("product_linkss.csv")

In [49]:
product_links_df

,Product Links
0,https://goldentech.com.sa/en/laptop-microsoft-...
1,https://goldentech.com.sa/en/laptop-acer-trave...
2,https://goldentech.com.sa/en/laptop-hp-15-inte...
3,https://goldentech.com.sa/en/laptop-apple-macb...
4,https://goldentech.com.sa/en/laptop-acer-aspir...
...,...
310,https://goldentech.com.sa/en/laptop-apple-macb...
311,https://goldentech.com.sa/en/laptop-lenovo-ide...
312,https://goldentech.com.sa/en/laptop-hp-250-g10...
313,https://goldentech.com.sa/en/microsoft-surface...


In [75]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Set up Selenium WebDriver
driver = webdriver.Chrome()

# Load the CSV file containing the product links
product_links_df = pd.read_csv("product_linkss.csv")
product_links = product_links_df["Product Links"].tolist()  # List of product links

# Initialize an empty list to store product details
product_data = []
# Iterate through each product link
for link in product_links:
    driver.get(link)
    time.sleep(2)  # Wait for the page to load

    # Handle cookie consent overlay
    try:
        # Wait for the "Accept all" button to be visible and clickable
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".cmpboxbtnyes"))
        )
        # Click the "Accept all" button to close the cookie consent overlay
        accept_button.click()
        time.sleep(2)  # Wait for the overlay to disappear
    except Exception as e:
        print(f"No cookie overlay found for {link}: {e}")

   

    # # Extract the product title
    # try:
    #     product_title = driver.find_element(By.CSS_SELECTOR, "h2.product-title__title").text.strip()
    # except Exception as e:
    #     product_title = "Not available"
    #     print(f"Error extracting product title for {link}: {e}")

    # Extract the product price
    # try:
    #     # Wait for the special price to be visible
    #     special_price_element = WebDriverWait(driver, 10).until(
    #         EC.visibility_of_element_located((By.CSS_SELECTOR, ".special-price .price"))
    #     )
    #     special_price = special_price_element.text.strip()
    # except Exception as e:
    #     product_price = "Not available"
    #     print(f"Error extracting product price for {link}: {e}")

    # Scroll to the bottom of the page after clicking the "Show more" button
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Wait for the page to fully load and new content to appear
    try:
        # Wait for the special price to be visible
        special_price_element = WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".special-price .price"))
        )
        special_price = special_price_element.text.strip()
    except Exception as e:
        product_price = "Not available"
        print(f"Error extracting product price for {link}: {e}")
        
    # Find the product specification table under 'More Information'
    try:
        table = driver.find_element(By.CSS_SELECTOR, "table.data.table.additional-attributes")
        rows = table.find_elements(By.TAG_NAME, "tr")  # Get all rows in the table

        # Dictionary to hold the details for each product
        product_info = {
            # "Product Title": product_title,
            "Product Price": special_price
        }

        # Iterate through each row in the table and extract the column name and value
        for row in rows:
            try:
                # Extract column name (th) and value (td)
                column_name = row.find_element(By.CSS_SELECTOR, "th.col.label").text.strip()
                value = row.find_element(By.CSS_SELECTOR, "td.col.data").text.strip()

                # Add the extracted data to the product_info dictionary
                product_info[column_name] = value
            except Exception as e:
                print(f"Skipping row due to missing data or invalid structure: {e}")

        # Append the product info to the product_data list
        product_data.append(product_info)

    except Exception as e:
        print(f"Error extracting specifications table for {link}: {e}")

# Convert the list of dictionaries to a DataFrame
product_details_df = pd.DataFrame(product_data)

# Save the DataFrame to a CSV file
product_details_df.to_csv("GoldenTechnology.csv", index=False)

# Print confirmation
print(f"Saved product details to 'product_details.csv'")

# Close the driver
driver.quit()



No cookie overlay found for https://goldentech.com.sa/en/laptop-microsoft-surface-pro-9-qix-00025.html: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF711022775+28773]
	(No symbol) [0x00007FF710F8AFB0]
	(No symbol) [0x00007FF710E2552A]
	(No symbol) [0x00007FF710E78EAE]
	(No symbol) [0x00007FF710E7919C]
	(No symbol) [0x00007FF710EC23F7]
	(No symbol) [0x00007FF710E9EFDF]
	(No symbol) [0x00007FF710EBF1A4]
	(No symbol) [0x00007FF710E9ED43]
	(No symbol) [0x00007FF710E6A548]
	(No symbol) [0x00007FF710E6B6B1]
	GetHandleVerifier [0x00007FF71134F45D+3358029]
	GetHandleVerifier [0x00007FF71136430D+3443709]
	GetHandleVerifier [0x00007FF7113583FD+3394797]
	GetHandleVerifier [0x00007FF7110E929B+842635]
	(No symbol) [0x00007FF710F9654F]
	(No symbol) [0x00007FF710F91FA4]
	(No symbol) [0x00007FF710F9213D]
	(No symbol) [0x00007FF710F81629]
	BaseThreadInitThunk [0x00007FFF1B8F259D+29]
	RtlUserThreadStart [0x00007FFF1BD8AF38+40]

No cookie overlay found for https://goldentech.com.sa/en/laptop-acer-tr

In [76]:
product_details_df

,Product Price,Product Type,Model Number,Processor Type,Processor Speed,RAM,Storage Capacity,Graphics Card,Screen Size,Display Resolution,...,Sync Technology,Keyboard type,Microsoft Office,Mobile broadband,High Dynamic Range,User Guide,Neural Processing,Fingerprint Type,Response Time,Microphone Type
0,"6,799 SR",2-in-1 Laptop,Microsoft Surface Pro 9,Intel Core i7-1265U (12th Gen) Evo Platform,1.2 GHz,16GB LPDDR5,512GB SSD M.2,Intel Iris Xe Graphics,13 inch,2880X1920 (267 PPI),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,899 SR",Business Laptop,Acer Travelmate TMP215-53-596G,Intel Core i5-1135G7 (11th Gen),2.4 GHz,8GB DDR4,512GB SSD M.2,Intel Iris Xe Graphics,15.6 inch,1920x1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,149 SR",Laptop,HP 15-fd0021nx,Intel Core i3-1315U (13th Gen),1.2 GHz,4GB DDR4 3200MHz,256GB SSD M.2,Intel Iris Xe Graphics,15.6 inch,1920x1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"4,499 SR",Laptop,MacBook Air M2 15.3 2023,Apple M2 chip,8-Core CPU,8GB Ram,256GB SSD,10-core GPU,15.3 inch,Liquid Retina 2880x1864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"1,999 SR",Laptop,Acer Aspire 3 A315-58-74GF,Intel Core i7-1165G7 (11th Gen),2.8 GHz,8GB DDR4,512GB SSD M.2,Intel Iris Xe Graphics,15.6 inch,1920x1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,"2,699 SR",Laptop,MacBook Air M1 2020,Apple M1 chip,8-Core CPU,8GB Ram,256GB SSD,7-core GPU,13.3 inch,Retina Display 2560x1600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,994 SR,Laptop,Lenovo IdeaPad 1 15IAU7,Intel Core i3-1215U (12th Gen),0.9 GHz,4GB DDR4 3200MHz,512GB SSD M.2,Intel UHD Graphics,15.6 inch,1366 x 768 HD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,"1,199 SR",Laptop,HP 250 G10,Intel Core i3-1315U (13th Gen),0.9 GHz,4GB DDR4 3200MHz,256GB SSD M.2,Intel UHD Graphics,15.6 inch,1920x1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,"1,149 SR",2-in-1 Laptop,Microsoft Surface Go 2,Intel Pentium 4425Y,1.7 GHz,4GB Ram,64GB eMMC,Intel UHD Graphics 615,"10.5"" inch",1920 x 1280 (220 PPI),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
